In [175]:
import laspy
import visualize
import cluster_dbscan
from tqdm import tqdm
import torch
import numpy as np
from classifier_net import classifier_network, create_network
import matplotlib.pyplot as plt
from PIL import Image
import db_settings
import geopandas
from scipy.spatial import ConvexHull
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.affinity import scale
import pandas

In [176]:
# map to 
db = db_settings.db()
#kurve:
#las_file = 'Demo/Köln/3dm_32_350_5648_1_nw.las'
#las_name = "3dm_32_350_5648_1_nw"
#geht auch, das feld, vielelicht für CHM?:
#las_file = 'Demo/Köln/3dm_32_351_5654_1_nw.las'
#las_name = "3dm_32_351_5654_1_nw"
las_file = 'Demo/color/vegetation_3dm_32_335_5728_1_nw.las'
las_name = "3dm_32_335_5728_1_nw"
req = """SELECT ID, NUMBER_OF_TREES FROM lidar_proj.LIDAR_FILES WHERE FILE_NAME = \'"""+ las_name+"""\'"""
print(req)
lidar_file_number = db.export_to_pandas(req)
req ="""SELECT * FROM lidar_proj.trees WHERE city_ID=""" + str(2) +""" and LIDAR_FILE_ID="""+str(lidar_file_number['ID'].values[0])+""" ORDER BY id"""
print(req)
trees =db.export_to_pandas(req)
print(trees)


SELECT ID, NUMBER_OF_TREES FROM lidar_proj.LIDAR_FILES WHERE FILE_NAME = '3dm_32_335_5728_1_nw'
SELECT * FROM lidar_proj.trees WHERE city_ID=2 and LIDAR_FILE_ID=913 ORDER BY id
Empty DataFrame
Columns: [ID, X, Y, CITY_ID, ID_GATTUNG, LIDAR_FILE_ID]
Index: []


In [177]:
#las_file = 'Demo/3dm_32_362_5716_1_nw.las'
#las_file = 'Demo/3dm_32_362_5710_1_nw.las'
#las_file = 'Demo/3dm_32_370_5709_1_nw_classified.laz'


las_cluster =laspy.read(las_file)
las_color =laspy.read(las_file)
cluster_dict,color_dict, indices_dict = cluster_dbscan.cluster_las(las_cluster)

time:  2022-07-26 00:05:39.730340
selecting points to cluster by classification
clustering


100%|██████████| 941428/941428 [00:03<00:00, 300619.90it/s]


building dictionary:


100%|██████████| 2877/2877 [00:07<00:00, 409.98it/s]


In [178]:
hull_dict = {}

for key in cluster_dict.keys():
    tree = np.array(cluster_dict[key]).transpose((1, 0))
    x = tree[0]
    y = tree[1]    
    points_2d = np.array([x,y]).transpose()
    if len(points_2d) > 150:
                hull = ConvexHull(points_2d)
                poly = Polygon(points_2d[hull.vertices])
                poly = scale(poly, xfact=1.1,yfact=1.1)
                hull_dict[key] = poly
                



In [169]:
#trees['coords'] = np.array(zip(trees['X'],trees['Y']))
#trees['coords'] = trees['coords'].apply(Point)
#trees_df = geopandas.GeoDataFrame(trees, geometry='coords', crs="EPSG:25832")
#print(trees)                


In [170]:
#hulls_df = pandas.DataFrame({'HULL_DICT_KEY': hull_dict.keys(), 'coords': hull_dict.values()})
#hulls_df = geopandas.GeoDataFrame(hulls_df, geometry='coords', crs="EPSG:25832")
#print("calculating joins")
#intersections = geopandas.tools.sjoin(trees_df,hulls_df, op="intersects", how='inner') #intersects
#print(intersections)

In [180]:
#print(color_dict[0])
#k = np.unique(np.array(intersections.HULL_DICT_KEY))
#print(len(k))
for key in tqdm(cluster_dict.keys()):
    for i in indices_dict[key]:
        las_cluster.points['red'][i] = color_dict[key][0]
        las_cluster.points['green'][i]= color_dict[key][1]
        las_cluster.points['blue'][i]= color_dict[key][2]

100%|██████████| 2876/2876 [00:08<00:00, 349.94it/s]


In [182]:
visualize.plot_las_3d(las_cluster)

[[0.38039216 0.35686275 0.36470588]
 [0.3372549  0.3372549  0.3372549 ]
 [0.4        0.37647059 0.38431373]
 ...
 [0.29411765 0.38823529 0.33333333]
 [0.29019608 0.38431373 0.32941176]
 [0.27058824 0.36470588 0.30196078]]


In [160]:
#visualize.plot_las_3d(las_color)

In [204]:
normalized_dict = {}
for key in tqdm(cluster_dict.keys()):
        points = np.array(cluster_dict[key])
        
        x , y, z = points.transpose()
        factor = max(np.max(x),np.max(y), np.max(z))
        
        x = np.array(x)
        x = (x - np.min(x))/factor
        y = np.array(y)
        y = (y - np.min(y))/factor
        z = np.array(z)
        z = (z - np.min(z))/factor
        points = np.array([x,y,z]).transpose()
        normalized_dict[key] = points
print(factor)

100%|██████████| 2876/2876 [00:00<00:00, 21174.65it/s]

5728901.58


In [244]:
from monai.transforms import (Compose, AsChannelFirst, AddChannel, Resize,ScaleIntensity, RandGaussianSharpen, SpatialPad, ToTensor, Zoom, LoadImage,RandScaleCrop,OneOf, RandHistogramShift,RandCoarseDropout,AddChannel,RandFlip,RandZoom, RandRotate,EnsureType,NormalizeIntensity, Flip,  RandAdjustContrast, RandGaussianSmooth,RandBiasField,RandStdShiftIntensity,  GaussianSharpen,RandGaussianNoise, HistogramNormalize, RandGibbsNoise, RandKSpaceSpikeNoise, Rotate, compose)


network_path ="0.41975308678768297_in_e_0_network"#"denseNetBinaryClass"
size = 128

trans_load_image = Compose([
             AsChannelFirst(),
             ToTensor(),
             Resize((size, size)),

             EnsureType(),
             ])

fit_network_transform = Compose([
                                ToTensor(),
                                Resize((size, size)),
                                ScaleIntensity(), 
                                EnsureType(),             
                                AddChannel(),])
        

In [245]:
softmax = torch.nn.Softmax(dim=1)


predictions = {}
network = classifier_network(None, [1,2] )

network.net.load_state_dict(torch.load(network_path))
network.net.eval()

for key in tqdm(normalized_dict.keys()):
    list_of_points =normalized_dict[key]

    xz = np.zeros([256,256])        
    yz = np.zeros([256,256])
    xy = np.zeros([256,256])
    list_of_points = (list_of_points * 255).astype(int)
    for x,y ,z in list_of_points:
            xz[255 -z][x] += y
            yz[255- z][y] +=x
            xy[255 - y][x] += z
       
    img = np.array([xz, yz, xy]).transpose()
    m = np.max(img)
    img = (img/m) * 255
    img = Image.fromarray(np.uint8(img)).rotate(270)
    img = np.array(img)
    
    img = img/ 255

    #plt.imshow(img)
    #plt.show()

    
    image = trans_load_image(img)
    #image  = image.type(torch.int)
    image = fit_network_transform(image.to(network.device) )   
    pred = network.net(image.float()) 
    pred = softmax(pred)

    pred = pred.cpu().detach().numpy()
    predictions[key] = np.argmax(pred)
    if (np.argmax(pred) ==0):
        print(0)
   
    
    


cuda
pos weight: 1
2  classes to classify


100%|██████████| 2876/2876 [01:19<00:00, 36.32it/s]


In [227]:
color_dict_classification = color_dict
for key in predictions.keys():
    if predictions[key] == 1:
        color_dict_classification[key] = [48, 138, 70]
    else:
         color_dict_classification[key] = [0,0,0]# [0, 58, 34]

In [228]:
las_w_pred = las_color
for key in tqdm(cluster_dict.keys()):
    for i in indices_dict[key]:
        las_w_pred.points['red'][i] = color_dict_classification[key][0]
        las_w_pred.points['green'][i]= color_dict_classification[key][1]
        las_w_pred.points['blue'][i]= color_dict_classification[key][2]

100%|██████████| 2876/2876 [00:07<00:00, 365.91it/s]


In [229]:
visualize.plot_las_3d(las_w_pred)

[[0.38039216 0.35686275 0.36470588]
 [0.3372549  0.3372549  0.3372549 ]
 [0.4        0.37647059 0.38431373]
 ...
 [0.29411765 0.38823529 0.33333333]
 [0.29019608 0.38431373 0.32941176]
 [0.27058824 0.36470588 0.30196078]]


In [213]:
#las_w_pred.write("pred_las.laz")